In [37]:
import pandas as pd
import numpy as np 
from surprise import KNNBasic 
from surprise import Dataset 
from surprise.model_selection import cross_validate, train_test_split
from surprise import Dataset, NormalPredictor, Reader, SVD, accuracy
from collections import defaultdict

In [38]:
# -----------------------------
# 1️⃣ Load datasets
# -----------------------------
ratings = pd.read_csv('Ratings.csv')  # User-ID, ISBN, Book-Rating
books = pd.read_csv('Books.csv', dtype={'Year-Of-Publication': str})  # ISBN, Book-Title, etc.
users = pd.read_csv('Users.csv')  # User-ID, Location, Age

# Optional: inspect first few rows
ratings.head()


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [42]:
ratings = ratings.merge(users, on='User-ID', how='inner')

In [50]:
# Keep users with at least 50 ratings
user_counts = ratings['User-ID'].value_counts()
active_users = user_counts[user_counts >= 50].index
ratings_small = ratings[ratings['User-ID'].isin(active_users)]

# Keep books with at least 50 ratings
book_counts = ratings_small['ISBN'].value_counts()
popular_books = book_counts[book_counts >= 50].index
ratings_small = ratings_small[ratings_small['ISBN'].isin(popular_books)]

ratings_small = ratings_small[ratings_small['Book-Rating'].notna() & (ratings_small['Book-Rating'] > 0)]

print(f"Filtered dataset: {ratings_small.shape[0]} ratings, {len(active_users)} users, {len(popular_books)} books")


Filtered dataset: 30579 ratings, 3427 users, 1108 books


In [51]:
# -----------------------------
# 4️⃣ Define KNN algorithm
# -----------------------------
sim_options = {'name': 'cosine', 'user_based': False}  # User-based collaborative filtering
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the cosine similarity matrix...


ZeroDivisionError: float division

In [45]:
# -----------------------------
# 2️⃣2️ Prepare data for Surprise
# -----------------------------
# 'userID', 'itemID', 'rating' format required
reader = Reader(rating_scale=(0, 10))  # Adjust to the rating scale
data = Dataset.load_from_df(ratings_small[['User-ID', 'ISBN', 'Book-Rating']], reader)


In [46]:
# -----------------------------
# 3️⃣ Split train/test data
# -----------------------------
trainset, testset = train_test_split(data, test_size=0.25)

# Or use full trainset for final recommendations
trainset = data.build_full_trainset()

In [48]:
# -----------------------------
# 4️⃣ Define KNN algorithm
# -----------------------------
sim_options = {'name': 'cosine', 'user_based': False}  # User-based collaborative filtering
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the cosine similarity matrix...


ZeroDivisionError: float division